## Custom QC for motion energy

In [2]:
"""
Check sessions available with dlc data
@author: Ines
"""
import pandas as pd
import pickle 
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from one.api import ONE
from uuid import UUID
import json
from scipy.stats import zscore

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions/'
# functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/'
os.chdir(functions_path)
from video_functions import get_ME
from one_functions_generic import timeseries_PSTH

one = ONE(mode='remote')

### Get extended QC of all sessions

In [3]:
ext_qc_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
ext_qc = pickle.load(open(ext_qc_path + "dlc_left_extended_qc", "rb"))
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))

video_type = 'left'

In [4]:
reduce = ['eid', 'dlcLeft', 'videoLeft', '_videoLeft_focus', 
            '_videoLeft_position', '_videoLeft_framerate', '_videoLeft_pin_state', 
            '_videoLeft_brightness', '_videoLeft_resolution',
            '_videoLeft_timestamps', '_dlcLeft_if_mean_in_box', '_dlcLeft_lick_detection',
            '_videoLeft_camera_times', '_videoLeft_file_headers','_videoLeft_dropped_frames',
            '_videoLeft_wheel_alignment', '_dlcLeft_trace_all_nan', '_dlcLeft_time_trace_length_match',
            'ratio', 'miss_lick_count']

lick_custom = lick_custom[reduce]

In [5]:
interest = ['dlcLeft', 
       'videoLeft', '_videoLeft_focus', 
       '_videoLeft_position', '_videoLeft_framerate', '_videoLeft_pin_state', 
       '_videoLeft_brightness', '_videoLeft_resolution',
       '_videoLeft_timestamps', '_dlcLeft_if_mean_in_box', '_dlcLeft_lick_detection',
       '_videoLeft_camera_times', '_videoLeft_file_headers','_videoLeft_dropped_frames',
        '_videoLeft_wheel_alignment']

In [6]:
use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'WARNING', np.nan, 'PASS'])) &
                             (lick_custom['videoLeft'].isin(['FAIL', 'WARNING', np.nan, 'PASS'])) &
                             (lick_custom['ratio']>5)]

In [7]:
use_filter

,eid,dlcLeft,videoLeft,_videoLeft_focus,_videoLeft_position,_videoLeft_framerate,_videoLeft_pin_state,_videoLeft_brightness,_videoLeft_resolution,_videoLeft_timestamps,_dlcLeft_if_mean_in_box,_dlcLeft_lick_detection,_videoLeft_camera_times,_videoLeft_file_headers,_videoLeft_dropped_frames,_videoLeft_wheel_alignment,_dlcLeft_trace_all_nan,_dlcLeft_time_trace_length_match,ratio,miss_lick_count
0,56956777-dca5-468c-87cb-78150432cc57,FAIL,WARNING,True,True,"[True, 60.212]",PASS,True,True,True,False,True,"[True, 0]",True,None,PASS,False,False,44.32,15.0
0,7af49c00-63dd-4fed-b2e0-1b3bd945b20b,PASS,FAIL,True,True,"[False, 149.269]","[True, 80, 1]",True,True,True,True,True,"[True, 0]",True,"[True, 4, 1]","[True, -1]",True,True,65.03,35.0
0,f88d4dd4-ccd7-400e-9035-fa00be3bcfa8,PASS,FAIL,True,True,"[False, 149.269]","[True, 1, 1]",True,True,True,True,True,"[True, 1]",True,"[True, 8, 1]","[True, 1]",True,True,60.38,19.0
0,83d85891-bd75-4557-91b4-1cbb5f8bfc9d,PASS,FAIL,True,True,"[False, 149.269]","[True, 3, 1]",True,True,True,True,True,"[True, 0]",True,"[True, 10, 1]","[True, 1]",True,True,7.99,224.0
0,aec5d3cc-4bb2-4349-80a9-0395b76f04e2,PASS,FAIL,True,True,"[False, 149.269]","[True, 0, 1]",True,True,True,True,True,"[True, 1]",True,"[True, 11, 1]","[True, -1]",True,True,24.04,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,c557324b-b95d-414c-888f-6ee1329a2329,FAIL,FAIL,True,True,"[True, 60.126]",PASS,False,True,True,True,True,"[True, 0]",True,None,PASS,False,True,16.33,33.0
0,5d01d14e-aced-4465-8f8e-9a1c674f62ec,FAIL,FAIL,True,True,"[True, 60.126]","[True, 1, 0]",False,True,True,False,True,"[True, 0]",True,"[True, 0, 0]","[True, -8]",False,True,13.38,55.0
0,81a78eac-9d36-4f90-a73a-7eb3ad7f770b,FAIL,FAIL,True,True,"[True, 60.126]","[True, 0, 0]",False,True,True,False,True,"[True, 0]",True,"[True, 0, 0]","[True, 0]",False,True,9.3,76.0
0,626126d5-eecf-4e9b-900e-ec29a17ece07,FAIL,FAIL,True,True,"[True, 60.126]","[True, 0, 1]",False,True,True,True,True,"[True, 1]",True,"[True, 1, 1]","[True, -16]",False,True,16.11,50.0


In [ ]:
save_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/Figures/notdlc_custom/'
save_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/Figures/'

files = os.listdir(save_path)

use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'CRITICAL', np.nan]))]
use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'WARNING', np.nan, 'PASS'])) &
                             (lick_custom['videoLeft'].isin(['FAIL', 'WARNING', np.nan, 'PASS'])) &
                             (lick_custom['ratio']>5)]

eids = use_filter['eid'].unique()
for e, session in enumerate(eids):
        filename = str(session + 'whisker_psth_-12_feedback.png')
        if filename not in files:
                try:
                        leftVideo_qc = list(lick_custom.loc[lick_custom['eid']==session, 'videoLeft'])[0]
                        leftDLC_qc = list(lick_custom.loc[lick_custom['eid']==session, 'dlcLeft'])[0]
                        missing_licks = list(lick_custom.loc[lick_custom['eid']==session, 'miss_lick_count'])[0]
                        lick_ratio = list(lick_custom.loc[lick_custom['eid']==session, 'ratio'])[0]
                        
                        #if list(lick_custom.loc[lick_custom['eid']==session, '_videoLeft_position'])[0] in [False, 'FAIL']:
                                
                        me_times, motion_energy = get_ME(one, session, video_type)       
                        m = zscore(motion_energy,nan_policy='omit') 
                        
                        t_init = 1
                        t_end = 2
                        event = 'stimOn_times'
                        event = 'feedback_times'

                        session_trials = one.load_object(session, obj='trials', namespace='ibl')
                        session_trials = session_trials.to_df() 
                        
                        # Align data for PSTH
                        ME_left = timeseries_PSTH(me_times, m, session_trials, event, t_init, t_end, subtract_baseline=False)
                        
                        # Plot and save

                        fig, ax = plt.subplots(nrows=1, ncols=1, sharex=False, sharey=True, figsize=[7, 5])
                        plt.rc('font', size=12)

                        this_data = ME_left.reset_index()
                        this_data['smooth_value'] = this_data['value'].rolling(3, min_periods=1).mean()
                        this_data = this_data.loc[this_data['choice'].isin([1., -1.])]
                        sns.lineplot(x='variable', y='smooth_value', style='feedback', data=this_data, ax = ax)
                        #plt.hlines(0, -0.5, 1, color='gray', linestyles='--')
                        plt.vlines(0, -.75, 1.5, color='gray', linestyles='--')
                        ax.set_xlabel(' ')
                        ax.set_xlabel('Time from ' + event + ' (s)')
                        ax.set_ylabel('z-scored motion energy')
                        ax.set_ylabel('z-scored motion energy')
                        ax.set_title('Video_qc: ' + str(leftVideo_qc)+' DLC_qc: ' + str(leftDLC_qc)+ 
                                '\n Lick/trial ratio: '+str(lick_ratio)+
                                '\n Lick-null trials: '+str(missing_licks))
                        plt.tight_layout()

                        # Save the plot as a PNG file
                        plt.savefig(str(save_path + session + 'whisker_psth_-12_feedback.png'), format='png')

                        # Display the plot
                        plt.show()
                        
                        # for v in interest:
                        #         var_interest = list(ext_qc.loc[ext_qc['eid']==session, v])[0]
                        #         print(str(v + ' ' +str(var_interest)))
                        # else:
                        #         print(list(lick_custom.loc[lick_custom['eid']==session, '_videoLeft_position'])[0])
                except:
                        
                        print(session)

4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b
7082d8ff-255a-47d7-a839-bf093483ec30
ac7d3064-7f09-48a3-88d2-e86a4eb86461
3dd347df-f14e-40d5-9ff2-9c49f84d2157
a19c7a3a-7261-42ce-95d5-1f4ca46007ed
931a70ae-90ee-448e-bedb-9d41f3eda647
cf43dbb1-6992-40ec-a5f9-e8e838d0f643
4a45c8ba-db6f-4f11-9403-56e06a33dfa4
10fac7a1-919d-4ca7-83c4-4675bd8d9416
cea755db-4eee-4138-bdd6-fc23a572f5a1
493170a6-fd94-4ee4-884f-cc018c17eeb9
9e9c6fc0-4769-4d83-9ea4-b59a1230510e
a4a74102-2af5-45dc-9e41-ef7f5aed88be
4d8c7767-981c-4347-8e5e-5d5fffe38534
eebacd5a-7dcd-4ba6-9dff-ec2a4d2f19e0
d832d9f7-c96a-4f63-8921-516ba4a7b61f
dcceebe5-4589-44df-a1c1-9fa33e779727
4ddb8a95-788b-48d0-8a0a-66c7c796da96
f8041c1e-5ef4-4ae6-afec-ed82d7a74dc1
65f5c9b4-4440-48b9-b914-c593a5184a18
25d1920e-a2af-4b6c-9f2e-fc6c65576544
c728f6fd-58e2-448d-aefb-a72c637b604c
f304211a-81b1-446f-a435-25e589fe3a5a
73918ae1-e4fd-4c18-b132-00cb555b1ad2
5adab0b7-dfd0-467d-b09d-43cb7ca5d59c
49e0ab27-827a-4c91-bcaa-97eea27a1b8d
edd22318-216c-44ff-bc24-49ce8be78374


Bugged sessions

In [ ]:
'4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b' # missing ME, but should be there according to alyx
'cf43dbb1-6992-40ec-a5f9-e8e838d0f643' # 
'4a45c8ba-db6f-4f11-9403-56e06a33dfa4'
'10fac7a1-919d-4ca7-83c4-4675bd8d9416'
'493170a6-fd94-4ee4-884f-cc018c17eeb9'
'edd22318-216c-44ff-bc24-49ce8be78374'
'73918ae1-e4fd-4c18-b132-00cb555b1ad2'
'25d1920e-a2af-4b6c-9f2e-fc6c65576544'
'eebacd5a-7dcd-4ba6-9dff-ec2a4d2f19e0'
'a4a74102-2af5-45dc-9e41-ef7f5aed88be'
'493170a6-fd94-4ee4-884f-cc018c17eeb9'